In [1]:
from SimPEG import *
from simpegAIP.FD import AFEMIPProblem_b
from SimPEG import EM
from pymatsolver import MumpsSolver
from scipy.constants import mu_0

In [ ]:
cs, ncx, ncz, npad = 25., 25, 20, 15
hx = [(cs,ncx), (cs,npad,1.3)]
hz = [(cs,npad,-1.3), (cs,ncz), (cs,npad,1.3)]
mesh = Mesh.CylMesh([hx,1,hz], '00C')    
sighalf = 1e-2
siginf = np.ones(mesh.nC)*1e-8
siginf[mesh.gridCC[:,-1]<0.] = sighalf
eta = np.ones(mesh.nC)*0.2
tau = np.ones(mesh.nC)*0.005
c = np.ones(mesh.nC)*0.7
m = np.r_[siginf, eta, tau, c]
mapping = ColeColeTimeMap(mesh)
prob = ProblemATEMIP_b(mesh, mapping=mapping)
waveformType="STEPOFF"
prob.timeSteps = [(1e-6, 20), (1e-5, 20), (1e-4, 20), (1e-3, 10)]
tobs = np.logspace(-4, -2, 21)
offset = 50.
rx = EM.TDEM.RxTDEM(np.array([[offset, 0., 0.]]), tobs, "bz")
# src = EM.TDEM.SrcTDEM_CircularLoop_MVP([rx], np.array([[0., 0., 0.]]), 13., waveformType)
src = EM.TDEM.SrcTDEM_VMD_MVP([rx], np.array([[0., 0., 0.]]), waveformType=waveformType)
survey = EM.TDEM.SurveyTDEM([src])
prob.Solver = MumpsSolver
prob.pair(survey)